In [ ]:
import arcpy
import os
from arcpy import env
import pandas as pd
import string

arcpy.env.overwriteOutput = True

### Examine parcel data
arcpy.env.workspace = r"C:\Users\vpavao\Documents\L3_data"
parcel_fcs = arcpy.ListFeatureClasses()
#print(parcel_fcs)

###Set data paths
east_parc = r"C:\Users\vpavao\Documents\L3_data\L3_TAXPAR_POLY_ASSESS_EAST.shp"
west_parc = r"C:\Users\vpavao\Documents\L3_data\L3_TAXPAR_POLY_ASSESS_WEST.shp"
structs = r"C:\Users\vpavao\Documents\structures\STRUCTURES_POLY.shp"
pws_service_data = r"C:\Users\vpavao\Documents\PWS_boundaries\PWS_WATER_SERVICE_AREA_COMM_POLY.shp"
pws_source_data = r"C:\Users\vpavao\Documents\PWS\PWSDEP_PT.shp"
townsshp = r"C:\Users\vpavao\Documents\Towns\TOWNSSURVEY_POLY.shp"
major_basins_data = r"C:\Users\vpavao\Documents\Major_Basins\MAJBAS_POLY.shp"
subbasins = r"C:\Users\vpavao\Documents\Subbasins\SUBBASINS_POLY.shp"


# Code below can be used to examine field information for parcels data
'''
fields = arcpy.ListFields(east_parc)
for field in fields:
    print(f"{field.name} has a type of {field.type} with a length of {field.length}")
'''

#Import excel queries
res_file = pd.ExcelFile(r"C:\Users\vpavao\Documents\Task3\Appendix N-queries\residential_use_codes.xlsx")
unc_file = pd.ExcelFile(r"C:\Users\vpavao\Documents\Task3\Appendix N-queries\uncertain_use_codes.xlsx")
dev_land_file = pd.ExcelFile(r"C:\Users\vpavao\Documents\Task3\Appendix N-queries\developable_land_use_codes.xlsx")
bldg_style_file = pd.ExcelFile(r"C:\Users\vpavao\Documents\Task3\Appendix N-queries\residential_styles.xlsx")
city_query_file = pd.ExcelFile(r"C:\Users\vpavao\Documents\Task3\Appendix N-queries\city_query.xlsx")

res_df = res_file.parse('resi use code query')
unc_df = unc_file.parse('uncertain use codes')
dev_df = dev_land_file.parse('developable_land')
bldg_df = bldg_style_file.parse('GREEN STYLES')
city_df = city_query_file.parse('Sheet1')

res_codes_raw = res_df["Residential use codes"].tolist()
unc_codes_raw = unc_df["Uncertain Use Codes"].tolist()
dev_codes_raw = dev_df['Residential "developable land" use codes'].tolist()
bldg_codes_raw = bldg_df["Residential styles to query in Step 7 of the methodology"].tolist()
city_query = city_df["TOWN"].tolist()
#print(city_query)

res_codes = []
for num in res_codes_raw:
    ###  Assume all codes from table are strings
    ### Remove doubly-wrapped quotes
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    res_codes.append(cd)

unc_codes = []
for num in unc_codes_raw:
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    unc_codes.append(cd)

dev_codes = []
for num in dev_codes_raw:
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    dev_codes.append(cd)

### Print use code lists
#print('Residential use codes', res_codes)
#print('Uncertain use codes', unc_codes)
#print('Developable land use codes', dev_codes)

###### Step 3 - Filter for residential parcels
layer_name_temp_east = "East_Parcels_Temp"
layer_name_temp_west = "West_Parcels_Temp"

arcpy.management.MakeFeatureLayer(east_parc, layer_name_temp_east)
arcpy.management.MakeFeatureLayer(west_parc, layer_name_temp_west)

### Function declaration for filtering parcels
def filter_study_parcels(layer, filterCodes, query_attr, output_name, switchSel, opposite_output_name): 
    for code in filterCodes:
        print(code)
        where_clause = query_attr + " = '{}'".format(code)
        arcpy.management.SelectLayerByAttribute(layer, "ADD_TO_SELECTION", where_clause)
    arcpy.management.CopyFeatures(layer, output_name)
    
    if(switchSel):
        arcpy.management.SelectLayerByAttribute(layer, "SWITCH_SELECTION")
        arcpy.management.CopyFeatures(layer, opposite_output_name)
    arcpy.management.SelectLayerByAttribute(layer, "CLEAR_SELECTION")

def filter_parcels_by_condition(layerName, outputLayerName, where_clause, switchSel, oppositeOutputName):
    arcpy.management.SelectLayerByAttribute(layerName, "ADD_TO_SELECTION", where_clause)
    arcpy.management.CopyFeatures(layerName, outputLayerName)
    
    if(switchSel):
        arcpy.management.SelectLayerByAttribute(layerName, "SWITCH_SELECTION")
        arcpy.management.CopyFeatures(layerName, oppositeOutputName)
    arcpy.management.SelectLayerByAttribute(layerName, "CLEAR_SELECTION")


### 3. Export parcels with residential use codes to separate files
res_output_name_east = "Residential-Parcels-east"
res_output_name_west = "Residential-Parcels-west"
res_working_output_east = "MA-Parcels-No-Residential-east"
res_working_output_west = "MA-Parcels-No-Residential-west"
filter_study_parcels(layer_name_temp_east, res_codes, "USE_CODE", res_output_name_east, True, res_working_output_east)
filter_study_parcels(layer_name_temp_west, res_codes, "USE_CODE", res_output_name_west, True, res_working_output_west)

### 4. Export parcels with uncertain use codes to separate files
unc_output_name_east = "Uncertain-Parcels-east"
unc_output_name_west = "Uncertain-Parcels-west"
unc_filter_output_east = "Uncertain-Parcels-with-ResArea-east"
unc_filter_output_west = "Uncertain-Parcels-with-ResArea-west"
unc_working_output_east = "Parcels-NotFilteredbyUncertainCode-east"
unc_working_output_west = "Parcels-NotFilteredbyUncertainCode-west" 
    

def generate_uncertain_layers():
    filter_study_parcels(res_working_output_east, unc_codes, "USE_CODE", unc_output_name_east, True, unc_working_output_east)
    filter_study_parcels(res_working_output_west, unc_codes, "USE_CODE", unc_output_name_west, True, unc_working_output_west)
    
    filter_parcels_by_condition(unc_output_name_east, unc_filter_output_east, "BLD_AREA - RES_AREA > 0", False, "")
    filter_parcels_by_condition(unc_output_name_west, unc_filter_output_west, "BLD_AREA - RES_AREA > 0", False, "")

    #Delete temporary layers
    #arcpy.management.Delete([unc_output_name_east, unc_output_name_west, unc_working_output_east, unc_nofilter_output_east, unc_working_output_west, unc_nofilter_output_west])
    
generate_uncertain_layers()

###5. Select all parcels in this new layer that completely contain a structure over 50 square
## meters by selecting polygon features within the MassGIS 2019 2D Structures layer, and export
## them to a new “developable w 2D struc” shapefile. Remove the parcels from the MA ParcelsWorking shapefile.
structs_lyr = "structures"
#arcpy.management.MakeFeatureLayer(structs, structs_lyr)

dev_filter_output_east = "MA-Dev-Parcels-east"
dev_filter_output_west = "MA-Dev-Parcels-west"
dev_output_name_east = "Developable-W-2D-structs-east"
dev_output_name_west = "Developable-W-2D-structs-west"

structs_over_50sqm = "structs_over_50sqm"

dev_code_working_output_east = "Parcels-NotFilteredbyDevelopableCode-east"
dev_code_working_output_west = "Parcels-NotFilteredbyDevelopableCode-west"

def query_developable_land(layer1, layer2, output_name):
    arcpy.management.SelectLayerByLocation(layer1, "COMPLETELY_CONTAINS", layer2, "","NEW_SELECTION")
    arcpy.management.CopyFeatures(layer1, output_name)
    arcpy.management.SelectLayerByAttribute(layer1, "CLEAR_SELECTION")

def generate_developable_layers():
    # Filter parcels for developable land use codes
    filter_study_parcels(unc_working_output_east, dev_codes, "USE_CODE", dev_filter_output_east, True, dev_code_working_output_east)
    filter_study_parcels(unc_working_output_west, dev_codes, "USE_CODE", dev_filter_output_west, True, dev_code_working_output_west)

    # Filter structures by area
    filter_parcels_by_condition(structs_lyr, structs_over_50sqm, "SHAPE_AREA > 50", False, "")

    #Select parcels with filtered structures
    query_developable_land(dev_filter_output_east, structs_over_50sqm, dev_output_name_east)
    query_developable_land(dev_filter_output_west, structs_over_50sqm, dev_output_name_west)

    #Delete temporary layers
    arcpy.management.Delete([
        dev_filter_output_east,
        dev_filter_output_west,
        structs_over_50sqm,
    ])
    
generate_developable_layers()

def delete_temp_layers():
    arcpy.management.Delete([
        layer_name_temp_east,
        layer_name_temp_west
    ])

delete_temp_layers()

def create_res_styles_layer(layer, styles, outputLyr, switchSel, oppositeOutputLyr):
    for style in styles:
        where_clause = "STYLE = '{}'".format(style)
        arcpy.management.SelectLayerByAttribute(layer, "ADD_TO_SELECTION", where_clause)
    arcpy.management.CopyFeatures(layer, outputLyr)

    if(switchSel):
        arcpy.management.SelectLayerByAttribute(layer, "SWITCH_SELECTION")
        arcpy.management.CopyFeatures(layer, oppositeOutputLyr)
    arcpy.management.SelectLayerByAttribute(layer, "CLEAR_SELECTION")

building_style_layer_east = "Residential-Style-Parcels-east"
building_style_layer_west = "Residential-Style-Parcels-west"
final_parcels_east = "MA-Working-Parcels-east"
final_parcels_west = "MA-Working-Parcels-west"

#create_res_styles_layer(dev_code_working_output_east, bldg_codes_raw, building_style_layer_east, True, final_parcels_east)
#create_res_styles_layer(dev_code_working_output_west, bldg_codes_raw, building_style_layer_west, True, final_parcels_west)

ma_resi_parcels = "MA-Resi-Parcels"

def merge_east_west():
    arcpy.management.Merge([dev_output_name_east, dev_output_name_west], "MA-Developable-Parcels")
    arcpy.management.Merge([unc_filter_output_east, unc_filter_output_west], "MA-Uncertain-Parcels")
    arcpy.management.Merge([res_output_name_east, res_output_name_west], "MA-Residential-Parcels")
    arcpy.management.Merge([building_style_layer_east, building_style_layer_west], "MA-Res-Style-Parcels")
    arcpy.management.Merge(["MA-Developable-Parcels", "MA-Uncertain-Parcels", "MA-Residential-Parcels", "MA-Res-Style-Parcels"], ma_resi_parcels)
    arcpy.management.Delete(["MA-Developable-Parcels", "MA-Uncertain-Parcels", "MA-Residential-Parcels", "MA-Res-Style-Parcels"])

#merge_east_west()

### 14. Subtract parcels inside PWS service areas

def query_from_pws(layer1, layer2, output_name):
    arcpy.management.SelectLayerByLocation(layer1, "INTERSECT", layer2,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(layer1,"SWITCH_SELECTION")
    arcpy.management.CopyFeatures(layer1, output_name)
    arcpy.management.SelectLayerByAttribute(layer1, "CLEAR_SELECTION")

pws_service_areas = "PWS_Service_Areas"
parcels_outside_pws_output = "15_Resi_Parcels_No_PWS_Zones"
#arcpy.management.MakeFeatureLayer(pws_service_data, pws_service_areas)
#query_from_pws(ma_resi_parcels, pws_service_areas, parcels_outside_pws_output)

#### 15. Subtract all parcels in city query list - this step should not be performed until further notice
'''
resi_parcels_minus_cities_list = "15-Resi-Parcels-Minus-Cities-List"
filter_study_parcels(parcels_outside_pws_output, city_query, "CITY", "Resi-Parcels-In-CitiesList", True, resi_parcels_minus_cities_list)
arcpy.management.Delete("Resi-Parcels-In-CitiesList")
'''

### 16. Delete parcels containing a PWS source
output_layer_16 = "16_Resi_Parcels_No_PWS_Points"

def filter_parcels_with_pws_source():
    pws_source_layer = "PWS_Source_Points"
    arcpy.management.MakeFeatureLayer(pws_source_data, pws_source_layer)
    arcpy.management.SelectLayerByLocation(parcels_outside_pws_output, "INTERSECT", pws_source_layer,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws_output,"SWITCH_SELECTION")
    arcpy.management.CopyFeatures(parcels_outside_pws_output, output_layer_16)
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws_output, "CLEAR_SELECTION")

#filter_parcels_with_pws_source()

### 17. Misc. queries for filtering study parcels. 
# These four queries were developed during the final QA/QC of the MA Resi Parcels layer. The
# queried parcel attributes were non-residential based on sampling parcels throughout the state.
        
output_layer_17 = "17_Resi_Parcels"
output_layer_17a = "17a_Filtered_Parcels"
output_layer_17b = "17b_Filtered_Parcels"
output_layer_17c = "17c_Filtered_Parcels"

def filter_misc_parcels():
    ###17a. Style = null; Bldg_Area = 0; Res_Area = 0; Address number = 0; and a structure from the 2019 MassGIS 2D Structures layer is not in the parcel.
    
    ### First select by attributes, and then select by location on that resulting layer
    arcpy.management.SelectLayerByLocation(output_layer_16, "INTERSECT", structs_lyr,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(output_layer_16, "SWITCH_SELECTION")
    
    where_clause_1 = "STYLE = '' AND BLD_AREA = 0 AND RES_AREA = 0 AND ADDR_NUM = '0'"
    arcpy.management.SelectLayerByAttribute(output_layer_16, "SUBSET_SELECTION", where_clause_1)
    arcpy.management.SelectLayerByAttribute(output_layer_16, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_16, output_layer_17a)
    arcpy.management.SelectLayerByAttribute(output_layer_16, "CLEAR_SELECTION")

    
    ###17b. Use_Code = 013, 014, 016, 0160, 017, 0170, 018, 0180, 959, 9590, 1333, or 071; Year_Built < 2015; and a structure from the 2019 MassGIS 2D Structures layer is not in the parcel.
    arcpy.management.SelectLayerByLocation(output_layer_17a, "INTERSECT", structs_lyr,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(output_layer_17a, "SWITCH_SELECTION")

    filterCodes = ['013', '014', '016', '0160', '017', '0170', '018', '0180', '959', '9590', '1333', '071']
    for code in filterCodes:
        print(code)
        where_clause_2 = "USE_CODE = '{}'".format(code)
        arcpy.management.SelectLayerByAttribute(output_layer_17a, "SUBSET_SELECTION", where_clause_2)
    arcpy.management.SelectLayerByAttribute(output_layer_17a, "SUBSET_SELECTION", "YEAR_BUILT < 2015")
    ### TODO Ask if YEAR_BUILT < 2015 selecting all parcels with 0 is intentional, should it follow a four-digit number?
    arcpy.management.SelectLayerByAttribute(output_layer_17a, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_17a, output_layer_17b)
    arcpy.management.SelectLayerByAttribute(output_layer_17a, "CLEAR_SELECTION")


    ###17c. Address number = 0; Res_Area = 0; and use_code does NOT = 101, 1010, 102, 1020, 103, 1040, 109, 1090, 130, 1300, 131, or 1310. 
    filterCodes2 = ['101', '1010', '102', '1020', '103', '1040', '109', '1090', '130', '1300', '131', '1310']
    for code in filterCodes2:
        print(code)
        where_clause_3 = "USE_CODE = '{}'".format(code)
        arcpy.management.SelectLayerByAttribute(output_layer_17b, "ADD_TO_SELECTION", where_clause_3)
    arcpy.management.SelectLayerByAttribute(output_layer_17b, "SWITCH_SELECTION")

    where_clause_4 = "RES_AREA = 0 AND ADDR_NUM = '0'"
    arcpy.management.SelectLayerByAttribute(output_layer_17b, "SUBSET_SELECTION", where_clause_4)

    arcpy.management.SelectLayerByAttribute(output_layer_17b, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_17b, output_layer_17c)
    arcpy.management.SelectLayerByAttribute(output_layer_17b, "CLEAR_SELECTION")
    
    ### 17d. Style = 'outbuildings'; and City does NOT = Rehoboth 
    where_clause_5 = "STYLE = 'Outbuildings' And NOT CITY = 'REHOBOTH'"
    arcpy.management.SelectLayerByAttribute(output_layer_17c, "NEW_SELECTION", where_clause_5)
    arcpy.management.SelectLayerByAttribute(output_layer_17c, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_17c, output_layer_17)
    arcpy.management.SelectLayerByAttribute(output_layer_17c, "CLEAR_SELECTION")
    
#filter_misc_parcels()

# Delete temporary layers for each step of 17 - comment out if you need to examine the output for each
#arcpy.management.Delete([output_layer_17a, output_layer_17b, output_layer_17c])

### 18. Dissolve the parcel layer by Major Basin, subbasin, and town and create a new layer for each of
#these three scales. Sum the “water-use” parcel values per dissolved unit in each layer. Name
#the four layers as follows: MA Resi Private Wells-parcels; MA Resi Private Wells-major basins;
#MA Resi Private Wells-subbasins; and MA Resi Private Wells-towns.
def dissolve_parcel_layers():
    towns_lyr = "Towns"
    major_basins_lyr = "Major_Basins"
    subbasins_lyr = "Subbasins"
    arcpy.management.MakeFeatureLayer(major_basins_data, major_basins_lyr)
    arcpy.management.MakeFeatureLayer(subbasins, subbasins_lyr)

    # Join parcels with major basins and then dissolve
    arcpy.analysis.SpatialJoin(output_layer_17, major_basins_lyr, output_layer_17 + "_MajBas_join", "JOIN_ONE_TO_ONE",
    "KEEP_ALL", "", "INTERSECT")
    arcpy.management.Dissolve(output_layer_17 + "_MajBas_join", "MA Resi Private Wells-major basins", "NAME")

    #Join parcels with sub-basins and then dissolve
    arcpy.analysis.SpatialJoin(output_layer_17, subbasins_lyr, output_layer_17 + "_SubBas_join", "JOIN_ONE_TO_ONE",
    "KEEP_ALL", "", "INTERSECT")
    arcpy.management.Dissolve(output_layer_17 + "_SubBas_join", "MA Resi Private Wells-subbasins", "SUB_NAME")
    
    arcpy.management.Dissolve(output_layer_17, "MA Resi Private Wells-towns", "CITY")
    arcpy.management.Delete([output_layer_17 + "_MajBas_join", output_layer_17 + "_SubBas_join", major_basins_lyr, subbasins_lyr])
    
#dissolve_parcel_layers()

001
0101
0102
0103
0104
0105
0107
0108
0109
010C
010D
010E
010F
010G
010H
010I
010J
010K
010L
010M
010R
010V
010Z
011
0110
0111
0112
0113
0114
011C
011I
011J
011Z
012
0120
0121
0123
012C
012O
013
0134
013C
013I
013M
013R
013V
014
0140
0143
0146
0147
014C
016
0160
0161
0163
0164
0167
0168
016R
017
0170
0171
0173
0174
0176
0178
017R
017V
018
0180
0181
0182
0183
0186
0187
018R
019
0190
0193
0197
021
0230
0302
0311
071
091
0961
1001
101
1010
1011
1012
1013
1014
1015
1016
1017
1018
101A
101B
101C
101F
101H
101M
101R
101V
102
1020
1021
1022
1023
1024
1027
1028
1029
102A
102C
102G
102H
102I
102P
102R
102U
102V
103
1030
1031
1032
1034
1035
103A
103C
103I
103P
103R
103V
104
1040
1040C
1041
1042
1043
1048
104A
104D
104M
104R
105
1050
1051
1053
1058
105C
105M
105R
106R
107
1070
1073
107V
108
1080
109
1090
1091
1092
1093
1094
1095
1098
1099
109C
109M
109R
109T
109V
110
111
1110
1111
1112
1113
1114
111C
111E
111J
111M
111R
111X
112
1120
1121
1123
1124
1125
1126
1129
112A
112B
112C
112I
112M
112R
11